Diwa Prasetyo


# Data Preparation

In [16]:
import pandas as pd
import tensorflow as tf
import re
import string
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential


In [17]:
df = pd.read_csv('/content/drive/MyDrive/BBC-News_Train.csv')
df

ArticleId                                               Text  \
0          1833  worldcom ex-boss launches defence lawyers defe...   
1           154  german business confidence slides german busin...   
2          1101  bbc poll indicates economic gloom citizens in ...   
3          1976  lifestyle  governs mobile choice  faster  bett...   
4           917  enron bosses in $168m payout eighteen former e...   
...         ...                                                ...   
1485        857  double eviction from big brother model caprice...   
1486        325  dj double act revamp chart show dj duo jk and ...   
1487       1590  weak dollar hits reuters revenues at media gro...   
1488       1587  apple ipod family expands market apple has exp...   
1489        538  santy worm makes unwelcome visit thousands of ...   

           Category  
0          business  
1          business  
2          business  
3              tech  
4          business  
...             ...  
1485  entertainment  
1486  entertainment  
1487       business  
1488           tech  
1489           tech  

[1490 rows x 3 columns]

In [18]:
df = df.drop(columns = 'ArticleId')
df

Text       Category
0     worldcom ex-boss launches defence lawyers defe...       business
1     german business confidence slides german busin...       business
2     bbc poll indicates economic gloom citizens in ...       business
3     lifestyle  governs mobile choice  faster  bett...           tech
4     enron bosses in $168m payout eighteen former e...       business
...                                                 ...            ...
1485  double eviction from big brother model caprice...  entertainment
1486  dj double act revamp chart show dj duo jk and ...  entertainment
1487  weak dollar hits reuters revenues at media gro...       business
1488  apple ipod family expands market apple has exp...           tech
1489  santy worm makes unwelcome visit thousands of ...           tech

[1490 rows x 2 columns]

In [19]:
category = pd.get_dummies(df.Category)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='Category')
df_baru = df_baru.replace({True:1, False:0})
df_baru

Text  business  \
0     worldcom ex-boss launches defence lawyers defe...         1   
1     german business confidence slides german busin...         1   
2     bbc poll indicates economic gloom citizens in ...         1   
3     lifestyle  governs mobile choice  faster  bett...         0   
4     enron bosses in $168m payout eighteen former e...         1   
...                                                 ...       ...   
1485  double eviction from big brother model caprice...         0   
1486  dj double act revamp chart show dj duo jk and ...         0   
1487  weak dollar hits reuters revenues at media gro...         1   
1488  apple ipod family expands market apple has exp...         0   
1489  santy worm makes unwelcome visit thousands of ...         0   

      entertainment  politics  sport  tech  
0                 0         0      0     0  
1                 0         0      0     0  
2                 0         0      0     0  
3                 0         0      0     1  
4                 0         0      0     0  
...             ...       ...    ...   ...  
1485              1         0      0     0  
1486              1         0      0     0  
1487              0         0      0     0  
1488              0         0      0     1  
1489              0         0      0     1  

[1490 rows x 6 columns]

In [20]:
df_baru["Text"] = df_baru["Text"].str.lower()
def remove_html_tags(text):

    html_tag_pattern = re.compile("<.*?>")
    cleaned_text = html_tag_pattern.sub(r"", text)
    return cleaned_text

df_baru["Text"] = df_baru["Text"].apply(remove_html_tags)
df_baru

Text  business  \
0     worldcom ex-boss launches defence lawyers defe...         1   
1     german business confidence slides german busin...         1   
2     bbc poll indicates economic gloom citizens in ...         1   
3     lifestyle  governs mobile choice  faster  bett...         0   
4     enron bosses in $168m payout eighteen former e...         1   
...                                                 ...       ...   
1485  double eviction from big brother model caprice...         0   
1486  dj double act revamp chart show dj duo jk and ...         0   
1487  weak dollar hits reuters revenues at media gro...         1   
1488  apple ipod family expands market apple has exp...         0   
1489  santy worm makes unwelcome visit thousands of ...         0   

      entertainment  politics  sport  tech  
0                 0         0      0     0  
1                 0         0      0     0  
2                 0         0      0     0  
3                 0         0      0     1  
4                 0         0      0     0  
...             ...       ...    ...   ...  
1485              1         0      0     0  
1486              1         0      0     0  
1487              0         0      0     0  
1488              0         0      0     1  
1489              0         0      0     1  

[1490 rows x 6 columns]

In [21]:
def remove_urls(text):

    url_pattern = re.compile(r"https?://\S+|www\S+")
    cleaned_text = url_pattern.sub(r"", text)
    return cleaned_text

df_baru["Text"] = df_baru["Text"].apply(remove_urls)

In [22]:
def remove_punctuations(text):
    punctuation_chars = string.punctuation
    cleaned_text = text.translate(str.maketrans("", "", punctuation_chars))
    return cleaned_text

df_baru["Text"] = df_baru["Text"].apply(remove_punctuations)
df_baru

Text  business  \
0     worldcom exboss launches defence lawyers defen...         1   
1     german business confidence slides german busin...         1   
2     bbc poll indicates economic gloom citizens in ...         1   
3     lifestyle  governs mobile choice  faster  bett...         0   
4     enron bosses in 168m payout eighteen former en...         1   
...                                                 ...       ...   
1485  double eviction from big brother model caprice...         0   
1486  dj double act revamp chart show dj duo jk and ...         0   
1487  weak dollar hits reuters revenues at media gro...         1   
1488  apple ipod family expands market apple has exp...         0   
1489  santy worm makes unwelcome visit thousands of ...         0   

      entertainment  politics  sport  tech  
0                 0         0      0     0  
1                 0         0      0     0  
2                 0         0      0     0  
3                 0         0      0     1  
4                 0         0      0     0  
...             ...       ...    ...   ...  
1485              1         0      0     0  
1486              1         0      0     0  
1487              0         0      0     0  
1488              0         0      0     1  
1489              0         0      0     1  

[1490 rows x 6 columns]

Data lebih dari 1000 sampel

In [23]:
text = df_baru['Text'].values
label = df_baru[['business', 'entertainment', 'politics', 'sport', 'tech']].values

# Modeling

In [24]:
#validation test 0.2 dari total
text_latih, text_test, label_latih, label_test = train_test_split(text,label, test_size=0.2)

In [25]:
#Penggunaan Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(text_latih)
tokenizer.fit_on_texts(text_test)

sekuens_latih = tokenizer.texts_to_sequences(text_latih)
sekuens_test = tokenizer.texts_to_sequences(text_test)

padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)

In [26]:
#Penggunaan model sequntial, Embedding dan LSTM
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [27]:
#menggunakan callbacks
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') and logs.get('val_accuracy')>0.85):
      print("\nAkurasi telah mencapai >90%!")
      self.model.stop_training = True
callbacks = myCallback()

In [28]:
#akurasi train dan validation diatas 85% ( bisa mencapai 90% jika dilatih lagi )
num_epochs = 50
history = model.fit(padded_latih, label_latih, epochs=num_epochs, batch_size = 25,
                    validation_data=(padded_test, label_test), verbose=2, callbacks=[callbacks])

Epoch 1/50
48/48 - 12s - loss: 1.6043 - accuracy: 0.2458 - val_loss: 1.5866 - val_accuracy: 0.3960 - 12s/epoch - 243ms/step
Epoch 2/50
48/48 - 8s - loss: 1.3934 - accuracy: 0.4765 - val_loss: 1.1961 - val_accuracy: 0.5201 - 8s/epoch - 174ms/step
Epoch 3/50
48/48 - 9s - loss: 0.9196 - accuracy: 0.6141 - val_loss: 0.9344 - val_accuracy: 0.6275 - 9s/epoch - 181ms/step
Epoch 4/50
48/48 - 6s - loss: 0.6464 - accuracy: 0.7055 - val_loss: 0.7532 - val_accuracy: 0.6879 - 6s/epoch - 130ms/step
Epoch 5/50
48/48 - 8s - loss: 0.3615 - accuracy: 0.8674 - val_loss: 0.5778 - val_accuracy: 0.8154 - 8s/epoch - 171ms/step
Epoch 6/50
48/48 - 6s - loss: 0.1163 - accuracy: 0.9614 - val_loss: 0.5827 - val_accuracy: 0.8289 - 6s/epoch - 128ms/step
Epoch 7/50
48/48 - 8s - loss: 0.0770 - accuracy: 0.9757 - val_loss: 0.6673 - val_accuracy: 0.8221 - 8s/epoch - 161ms/step
Epoch 8/50
48/48 - 5s - loss: 0.0244 - accuracy: 0.9958 - val_loss: 0.7991 - val_accuracy: 0.8154 - 5s/epoch - 113ms/step
Epoch 9/50
48/48 - 6s 

In [29]:
#memvisualisasikan plot
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()